<a href="https://colab.research.google.com/github/mrh-rakib/WASSA-2023-empathy/blob/main/main_EMP_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
from google.colab import drive
mount_path = '/content/drive'
drive.mount(mount_path)

Mounted at /content/drive


In [16]:
%cd $mount_path"/MyDrive/WASSA2023"

/content/drive/MyDrive/WASSA2023


In [17]:
essay_empathy = pd.read_csv("./essay_texts_all_preprocessed.csv", index_col=0)

In [18]:
essay_empathy.head(2)

,essay,emotion,conversation_id,article_id,empathy,distress,gender,education,race,age,...,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number
0,It breaks my heart to see people living in tho...,20,2,35,6.714286,6.714286,1.0,6.0,3.0,37.0,...,7.0,5.5,1.0,6.5,6.0,4.857,2.000,3.429,5.000,1
1,I wonder why there aren't more people trying t...,0,3,35,5.857143,6.000000,1.0,6.0,2.0,32.0,...,5.5,5.0,2.0,5.5,4.5,3.429,2.857,2.857,2.714,1


# Empathy

In [19]:
only_essay_empathy = essay_empathy[['essay', 'empathy']]

In [ ]:
# only_essay_empathy

In [12]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.3 MB/s eta 0:00:00


In [20]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_metric

In [21]:
from sklearn.metrics import mean_squared_error

In [22]:
hugging_dataset = Dataset.from_pandas(only_essay_empathy, preserve_index=False)
hugging_dataset = hugging_dataset.train_test_split(test_size = 0.2)

In [ ]:
# hugging_dataset

In [ ]:
# hugging_dataset['train']['essay'][:5]

In [23]:
checkpoint = "distilbert-base-uncased"

In [24]:
tokeniser = AutoTokenizer.from_pretrained(checkpoint)

In [31]:
only_essay_empathy

,essay,empathy
0,It breaks my heart to see people living in tho...,6.714286
1,I wonder why there aren't more people trying t...,5.857143
2,"After reading the article, you can't help but ...",1.000000
3,It is so sad that someone who had such an amaz...,6.000000
4,"From reading the article, it looks like the wo...",6.000000
...,...,...
787,I feel that this will become a national proble...,6.571429
788,The whole situation is sketchy. The wavering r...,3.571429
789,The death of a former aide to Russian Presiden...,5.428571
790,Everything about Russia really freaks me out. ...,6.000000


In [51]:
# checking length after tokenisation
max=0
for i in range (779):
  length = len(tokeniser(only_essay_empathy.iloc[i,0], padding=False, truncation=False)['input_ids'])
  if length>max:
    max=length

print(max)

194


In [47]:
def tokenise(sentence):
    return tokeniser(text=sentence["essay"], padding="longest", truncation=False)

In [48]:
tokenised_hugging_dataset = hugging_dataset.map(tokenise, batched=True)

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

In [27]:
tokenised_hugging_dataset

DatasetDict({
    train: Dataset({
        features: ['essay', 'empathy', 'input_ids', 'attention_mask'],
        num_rows: 623
    })
    test: Dataset({
        features: ['essay', 'empathy', 'input_ids', 'attention_mask'],
        num_rows: 156
    })
})

In [49]:
len(tokenised_hugging_dataset['train']['input_ids'][0])

183

In [ ]:
tokenised_hugging_dataset = tokenised_hugging_dataset.rename_column("essay", "text")
tokenised_hugging_dataset = tokenised_hugging_dataset.rename_column("empathy", "labels")

In [ ]:
tokenised_hugging_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 623
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 156
    })
})

In [ ]:
empathy_prediction = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit=2,
                                  save_strategy='no',
                                  load_best_model_at_end=False)

trainer = Trainer(
    model=empathy_prediction,
    args=training_args,
    train_dataset=tokenised_hugging_dataset["train"],
    eval_dataset=tokenised_hugging_dataset["test"],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rmse
1,3.473000,2.301469,1.517059
2,2.069100,2.182574,1.477353
3,1.353100,2.072373,1.439574


TrainOutput(global_step=117, training_loss=2.298410171117538, metrics={'train_runtime': 94.5657, 'train_samples_per_second': 19.764, 'train_steps_per_second': 1.237, 'total_flos': 247577152822272.0, 'train_loss': 2.298410171117538, 'epoch': 3.0})

In [ ]:
empathy_prediction.save_pretrained("model")
empathy_prediction.save_pretrained("tokenizer")

In [ ]:
raw_pred, _, _ = trainer.predict(tokenised_hugging_dataset["test"])

In [ ]:
y_pred = np.squeeze(raw_pred)

In [ ]:
import scipy.stats as st

In [ ]:
def pearson_r(y_true, y_pred):
    y_pred = y_pred.flatten()
    corr = st.pearsonr(y_true,y_pred)
    return corr[0]

In [ ]:
y_true = np.array(hugging_dataset["test"]["empathy"])

In [ ]:
y_true

array([1.57142857, 5.71428571, 5.14285714, 3.71428571, 6.71428571,
       4.42857143, 1.        , 1.        , 6.        , 6.71428571,
       1.14285714, 4.42857143, 2.85714286, 5.57142857, 5.85714286,
       5.28571429, 1.        , 5.28571429, 1.        , 6.14285714,
       6.        , 2.        , 1.57142857, 6.        , 3.71428571,
       4.28571429, 1.57142857, 2.        , 6.        , 6.        ,
       3.14285714, 1.28571429, 3.71428571, 5.57142857, 1.85714286,
       6.        , 3.71428571, 1.        , 6.        , 6.        ,
       3.85714286, 6.        , 5.71428571, 6.71428571, 3.85714286,
       5.85714286, 2.42857143, 5.42857143, 4.85714286, 4.42857143,
       1.85714286, 1.        , 2.28571429, 3.57142857, 3.57142857,
       6.        , 5.        , 2.        , 6.14285714, 4.85714286,
       4.42857143, 6.71428571, 6.        , 1.28571429, 3.85714286,
       5.85714286, 2.14285714, 5.28571429, 3.71428571, 6.        ,
       4.57142857, 5.42857143, 6.42857143, 1.        , 4.71428

In [ ]:
pearson_r(y_true=y_true, y_pred=y_pred)

0.6523309611055857